### Tabular 데이터 다루는 머신 러닝 파이프라인
1. 데이터 전처리
2. 피쳐 엔지니어일
3. 머신 러닝 모델 학습
4. 테스트 데이터예측 및 캐글 업로드 

### 1. 데이터 전처리
1. 제품변수의 결측값을 0으로 대체
    - 제품 보유 여부에 대한 정보가 없으면, 해당 제품은 없다고 가정
2. 훈련 데이터와 테스트 데이터 초합
    - 날짜 변수로 구분 가능 ( fecha_dato )
    - 동일한 24개의 고객 변수를 공유하며, 테스트 데이터에 없는 24개의 제품 변수를 0으로 채운다
3. 범주형, 수치형 데이터 전처리
    - 범주형은 .factorize() 통해 Label Encoding 
    - 데이터 타입이 Object로 표시된 수치형 데이터 ( e.g) age )는 .unique()를 통해 특이값들을 대체, 제거 후 정수형으로 전환
4. 모델 학습에 사용할 변수이름을 features 리스트에 미리 담자

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러온다.
print('Kaggle First job is start.. ')
trn = pd.read_csv('C:\\Users\\jaesang\\kaggle\\train_ver2.csv')
print('train set load finished..')

tst = pd.read_csv('C:\\Users\\jaesang\\kaggle\\test_ver2.csv')
print('test set load finished..')

## 데이터 전처리 ##

# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체한다.
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거한다.
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 제품 변수는 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list이다.
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

Kaggle First job is start.. 


MemoryError: 

### 2. 피쳐 엔지니어링

#### 모델에 학습할 파생 변수 생성

##### Baseline에서는 24개의 고객 변수 + 4개의 날짜 변수 기반 파생변수 + 24개의 Lag-1 변수 사용 

1. 4개의 날짜 변수 생성
    - fecha_alta 변수 = 고객이 첫 계약을 맺은 날짜
    - ult_fec_cli_1t 변수 = 고객이 마지막으로 1등급이었던 날짜
    - 두개의 변수에서 각각 연도와 월 정보추출
    > - 두개의 변수 간의 차이값을 파생변수 생성도 가능
    > - 졸업식, 방학등 특별한 날짜까지의 거리르 수치형으로 변수로도 사용 가능 

2. 결측값은 임시로 -99로 대체하자
    - 사이킷런에서 제공하는 ML 모델은 결측값을 입력값으로 받지않고 에러를 발생
    - Xgboost 모델은 결측값도 정상 입력값으로 받아, 데이터가 결측되었다는 것도 하나의 정보로 인식하고 모델 학습에 활용 
    - 이번 Baseline은 -99로 하자
    
3. 시계열 데이터는 고객의 과거 데이터 기반 다양한 파생 변수 만들 수 있음
    - 고객의 나이가 최근 3개월 동안 변동있는가 ( 생일이 지났음을 의미 ) - 이진 변수로 생성
    - 한달 전에 구매한 제품에 정보를 변수로 사용
    - 최근 평균 월급 계산도 가능
    
4. N개월 전에 금융 제품을 보유하고 있었는지 여부를 나타내는 Lag 변수가 좋은 파생 변수로 작용! 
    - 24개의 금융 제품 변수에 대하여 1개월, 2개월, 3개월 전 보유 여부를 변수로 활용
    - baseline에서는 1개월 전만 사용 ( lag-1 )

In [ ]:
# (피쳐 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

# (피쳐 엔지니어링) lag-1 데이터를 생성한다.
# 코드 2-12와 유사한 코드 흐름이다.

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]


### 3.  머신러링 모델 학습

#### 교차검증
> 경진대회에서 좋은 성적을얻기 위해 가장 중요한 것 ! 
> 캐글은 하루에 최대 5개의 예측 결과만 제출할 수 있고, 결과는 Public 리더보드에점수 공개
> 즉, 하루에 5번 밖에 확인할 수 없기에,다양한 실험을 제출로는 힘들 수 있다
> 그래서 !! 올바른 교차검증 방법으로 제한없이 아이디어를 실험하고, 성능 개선 여부를 확인해보자


- 훈련 셋 = 2015-01-28 ~ 2016-05-28, 총 1년 6개월치 데이터  
- 테스트 셋 = 2016-06-28, 미래 데이터
- 교차검증에서는 최신 날짜 2016-05-28를 검증 데이터로분리하고, 나머지는 훈련셋으로 사용
> baseline은 간소화하기 위해 2016-01-28~2016-04-28, 총 4개월 훈련 / 2016-05-28을 검증으로 사용



In [ ]:

## 모델 학습
# 학습을 위하여 데이터를 훈련, 테스트용으로 분리한다.
# 학습에는 2016-01-28 ~ 2016-04-28 데이터만 사용하고, 검증에는 2016-05-28 데이터를 사용한다.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

# 훈련, 검증 데이터로 분리한다. 
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]


#### 모델

##### XGBoost 모델 

- max_depth
> 트리 모델의 최대 깊이 = 깊을수록 복잡한 모델이지만 오버피팅 원인이 될 수 있음

- eta
> - learning_rate와 같은 개념 
> - 0~1사이의 값으로 너무 낮으면 학습이 잘 안될 수 있으며, 너무 크면 학습이 느려짐

- colsample-bytree
> -트리를 생성할 때, 훈련 데이터에서 변수를 샘플링해주는 비율 
> - 모든 트리는 전체 변수의 일부만 학습하여 서로의 약점을 보완해주는 모델 
> - 보통 0.6~0.9

- colsmaple_bylevel
> - 트리의 레벨 별로 훈련 데이터 변수를 샘플링해주는 비율
> - 보통 0.6~0.9

* 참고

> - 머신러닝을 처음하는 사람들은 모델의 최적화된 하이퍼파라미터찾는데 많은 시간 투자.. ( = 튜닝 작업 )
> - 물론 이 것도 좋지만, 시간 투자 대비효율을 생각하면 피처 엔지니어링에 더 투자하자

In [ ]:

# XGBoost 모델 parameter를 설정한다.
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

# 훈련, 검증 데이터를 XGBoost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# XGBoost 모델을 훈련 데이터로 학습한다!
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

# 학습한 모델을 저장한다.

import pickle
pickle.dump(model, open("'C:\\Users\\jaesang\\kaggle\\model\\xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit

### 교차검증

- 교차검증은 평가척도인 MAP@7 사용하여 성능 수준을 확인
> 경진대회에서 사용하는 평가 척도를 꼭 사용하자

- MAP@7 평가 척도는 최고 점수가 데이터에 따라 변동할 수 있다
- Baseline의 검증데이터에서는  MAP@7 최고 점수는 0.042663
- 1보다 낮은 이유는 검증 데이터의 모든 고객이 신규 구매를 하지 않기 때문
> - 100명의 고객 중 10명만이 신규구매를 헀다고 가정하면 그 10명을 정확히해도 결국 10%의 MAP를 받음 
> - 그러므로 MAP@7 최고 점수를 감안해서 성능 평가하자

In [ ]:
# MAP@7 평가 척도를 위한 준비작업이다.
# 고객 식별 번호를 추출한다.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
# 검증 데이터에서 신규 구매를 구한다.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# 고객별 신규 구매 정답 값을 add_vld_list에 저장하고, 총 count를 count_vld에 저장한다.
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

# 검증 데이터에서 얻을 수 있는 MAP@7 최고점을 미리 구한다. (0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

In [ ]:

# 검증 데이터에 대한 예측 값을 구한다.
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# 저번 달에 보유한 제품은 신규 구매가 불가하기 때문에, 확률값에서 미리 1을 빼준다
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

# 검증 데이터 예측 상위 7개를 추출한다.
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])
    
# 검증 데이터에서의 MAP@7 점수를 구한다. (0.036466)
print(mapk(add_vld_list, result_vld, 7, 0.0))

# XGBoost 모델을 전체 훈련 데이터로 재학습한다!
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
# XGBoost 모델 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

# 변수 중요도를 출력해본다. 예상하던 변수가 상위로 올라와 있는가?
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)


- Baseline모델은 MAP@ 0.036466 
- 검증 데이터 최고 점수가 0.042663
- 정확도는 (0.036466 / 0.042663 ) =0.85로 약 85%

### 4.  테스트 데이터 예측 및 캐글 업로드

- 교차검증에서 소중한 훈련 데이터의 일부를 도려내어 검증 데이터로 사용
- 테스트 데이터에 좋은 성능을 내기 위하여, 훈련, 검증 데이터를 합친 전체 데이터에 대하여 다시 XGBoost 모델을 학습
- XGBoost 모델 파라미터는 교차 검증에서 찾아낸 최적의 파라미터로 사용하되,
- XGBoost에서 사용하는 트리 개수를 늘어난 검증 데이터만큼증가

- 모델에서 변수 중요도 출력 ( XGBoost의 get_fscore() 
- 업로드

### 끝!! 맺음말

- Baseline 모델을 통해 데이터에 대한 이해도 깊어짐
- 머신러닝 파이프라인이 정상적으로 동작하는 거 확인 
- 이제 성능 개선하자!!

- 다시 강조하고자하는 내용은 '모델 튜닝'보다는 '피처 엔지니어링'에 많은 시간 투자하자
- 데이터에 대해서 심도 있게 고민하고 다양한 아이디어를 구현하고 실험하는 과정을 수행하면 , 많은 것을 얻을 수 잇을 것!

### Baseline 모델 요약

- 제공된 전체 훈련 데이터 중, 2016년도 데이터만을 학습에 사용 
- 파생 변수는 2개의 날짜 데이터의 연도, 월 추출
- 24개의 제품변수에 대한 lag-1변수만 사용 

- XGBoost 모델 학습 과정에서 MAP@7를 직접 사용할 수 없음
- XGBoost는 mlogloss를 통해 학습하였지만, 모델 파라미터를 선정하는 있어서는 
- 자체 구현한 mapk()을 통하여 검증 데이터에 MAP@7 점수 사용


